In [ ]:
import pandas as pd
import os

In [ ]:
filepath = '/media/sf_VBox_Shared/CaseLaw/2018-01-29-lido/derived/'
links_df = pd.read_csv(os.path.join(filepath, 'case-to-article-links.csv'))
case_nodes = pd.read_csv(os.path.join(filepath, 'case_nodes_simple_bimodal.csv'))
article_nodes = pd.read_csv(os.path.join(filepath, 'article_nodes_nodup_min5.csv'))

In [ ]:
article_nodes.head()

In [ ]:
case_nodes.head()

In [ ]:
links_df.head()

In [ ]:
links_enriched = links_df.merge(case_nodes, left_on='source', right_on='lido_id')

In [ ]:
links_enriched.shape

In [ ]:
links_enriched2 = links_enriched.merge(article_nodes, left_on='target', right_on='id')
links_enriched2.shape

In [ ]:
per_court_year = pd.DataFrame(links_enriched.groupby(['court', 'year', 'target']).count()['link_id'])
per_court_year = per_court_year.reset_index()
per_court_year = per_court_year.rename(columns={'link_id': 'count'})

In [ ]:
per_court =  pd.DataFrame(links_enriched.groupby(['court', 'target']).count()['link_id'])
per_court = per_court.reset_index()
per_court = per_court.rename(columns={'link_id': 'count', 'court':'source'})

In [ ]:
per_court_book =  pd.DataFrame(links_enriched2.groupby(['court', 'book']).count()['link_id'])
per_court_book = per_court_book.reset_index()
per_court_book.columns = ['source', 'target', 'weight']

In [ ]:
per_court.shape

In [ ]:
per_court.head()

In [ ]:
per_court.to_csv(os.path.join(filepath, 'court-to-article-links.csv'))

In [ ]:
per_court_min5 = per_court[per_court['count']>=5]
per_court_min5.shape

In [ ]:
per_court_min5.to_csv(os.path.join(filepath, 'court-to-article-links-min5.csv'))

In [ ]:
per_court_min5['source'].nunique(), per_court_min5['target'].nunique()

In [ ]:
per_court_book.shape

In [ ]:
per_court_book.to_csv(os.path.join(filepath, 'court-to-lawbook-links.csv'), index=False)

In [ ]:
per_court_book.sort_values(by='weight', ascending=False).head(29)

In [ ]:
from lxml import etree
courts_metadata_file = '/media/sf_VBox_Shared/CaseLaw/Instanties.xml'
courts_metadata = etree.parse(courts_metadata_file)

In [ ]:
code = 'HR'


In [ ]:
el.xpath('Naam')[0].text

In [ ]:
el.getchildren()

In [ ]:
el.tag

In [ ]:
courts = per_court_book['source'].unique()
print(courts)

In [ ]:
courts_metadata_dict = {}
for code in courts:
    metadata = courts_metadata.xpath("//Instantie[Afkorting='{}']".format(code))
    if(len(metadata)>0):
        courts_metadata_dict[code] = {}
        for el in metadata[0].getchildren():
            courts_metadata_dict[code][el.tag] = el.text

In [ ]:
courts_nodes = pd.DataFrame.from_dict(courts_metadata_dict, orient='index')

In [ ]:
courts_nodes = courts_nodes.rename(columns={'Naam': 'label', 'Afkorting': 'id'})

In [ ]:
courts_nodes.to_csv(os.path.join(filepath, 'court_nodes.csv'), index=False)